This notebook runs through the process of enabling NNCF in a simple semantic segmentation pipeline for training UNet on Camvid, and comparing results against the POT's quantization.

In [ ]:
!ls

Get the dataset:

In [ ]:
!git clone https://github.com/alexgkendall/SegNet-Tutorial data

Checkout the demo repo at the original, non-NNCF-enabled state:

In [ ]:
!git checkout baseline
!git show

Measure the baseline FP32 accuracy in PyTorch, produce an ONNX for future OpenVINO ingestion:

In [ ]:
!python3 main.py -m test export --resume unet_camvid.pth --data data/CamVid -b 1 --to-onnx unet_camvid.onnx

Evaluate the FP32 model on OpenVINO (accuracy and performance):

In [ ]:
pip install openvino

In [ ]:
pip install openvino-dev

In [ ]:
!mo --input_model unet_camvid.onnx --mean_values [99.603,103.329,105.6567] --scale_values [75.643,77.821,76.746] --reverse_input_channels

In [ ]:
!accuracy_check -c unet_camvid.yml -m unet_camvid.xml

In [ ]:
!benchmark_app -m unet_camvid.xml

Use POT to obtain an INT8 model:

In [ ]:
!pot -q accuracy_aware --max-drop 0.01 -m unet_camvid.xml -w unet_camvid.bin --ac-config unet_camvid.yml --output-dir pot_int8 --name unet_camvid_pot_aa_int8 -e -d

Measure performance for the INT8-POT model (the accuracy results are already visible in the POT output above) 

In [ ]:
!benchmark_app -m pot_int8/optimized/unet_camvid_pot_aa_int8.xml

Try to achieve better results with NNCF; install the NNCF package from PyPI

In [ ]:
pip install nncf

Integrate NNCF into the training pipeline; the line below checks out the demo repo state with NNCF integrated to allow for producing NNCF-compressed INT8 models. 5 lines of code (excluding import statements) and a simple .json config is enough for this integration.

In [ ]:
!git checkout with_nncf
!git diff baseline with_nncf

Perform compression-aware fine-tuning using NNCF, starting from the regular PyTorch checkpoint (unet_camvid.pth), for 10 epochs, picking the best result and exporting it into an INT8 ONNX file (unet_camvid_int8.onnx)

In [ ]:
!python3 main.py -m train test export --resume unet_camvid.pth --data data/CamVid -b 3 --epochs 10 --to-onnx unet_camvid_int8.onnx --nncf_config unet_camvid_int8.json

Convert the NNCF-INT8 ONNX file into the NNCF-INT8 IR for OpenVINO ingestion

In [ ]:
!mo --input_model unet_camvid_int8.onnx --mean_values [99.603,103.329,105.6567] --scale_values [75.643,77.821,76.746] --reverse_input_channels

Evaluate the NNCF-INT8 model in OpenVINO, accuracy and performance-wise:

In [ ]:
!accuracy_check -c unet_camvid.yml -m unet_camvid_int8.xml

In [ ]:
!benchmark_app -m unet_camvid_int8.xml